## Setup

In [32]:
# Import packages
import pandas as pd
from ds_common_utils.aux.io.snowflake_tools import SnowflakeTools
from datetime import datetime
import matplotlib.pyplot as plt

In [33]:
# Setup connection
con = SnowflakeTools(
    role="INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD",
    warehouse="INSIGHT_ANALYST_WH",
    database="BDWPRD_DE",
    schema="IA_MERCH_DE"
)

In [34]:
# Date range
start_date = "'2024-06-01'"
end_date = "'2025-05-31'"

## Data

In [35]:
# Set item range table name
table = 'RPC_item_range' + datetime.today().strftime('%Y_%m_%d')

In [36]:
# Create temp item table
con.execute_statement_from_sql_file(
    file='sql/0-item-range.sql',
    formatting={'table_name': table,
                'filter' : "AND item_sub_department_name = '400 RODENT PEST CONTROL' "})

snowflake_tools - 2025-07-07 12:59:46.477727+10:00 - Generating new token with 'INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD' role and 'SESSION:ROLE-ANY' scope...


snowflake_tools - 2025-07-07 12:59:52.913716+10:00 - Saved token with '2025-07-07 16:59:52+10:00' expiry...


## Demographic Segments (AU)

Overall 400 RPC

In [37]:
ds = con.read_sql_file_into_pandas(
    file="sql/demographic-segments-overall.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'filter' : "item_sub_department_name = '400 RODENT PEST CONTROL'"
    }
)

ds[["DEMOGRAPHIC_SEGMENT", "RANGE_SALES", "RANGE_SALES_SHARE", "SALES_INDEX"]]

,DEMOGRAPHIC_SEGMENT,RANGE_SALES,RANGE_SALES_SHARE,SALES_INDEX
0,Homeowner older kids,984296.77,0.082856,0.87
1,Homeowner younger kids,1885909.77,0.158752,0.84
2,Older homeowner,6042107.34,0.508612,1.22
3,Renter no kids,769299.70,0.064758,0.74
4,Renter with kids,721149.08,0.060705,1.00
5,Unclassifiable,103152.99,0.008683,1.27
6,Younger homeowner,1373690.09,0.115634,0.80


- Older homeowner over indexed
- Renter no kids under indexed

Class index

In [38]:
RPC_ds = con.read_sql_file_into_pandas(
    file="sql/1-demographic-segments.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'table_name': table,
        'level' : 'item_class_name',
        'country' : 'AU' 
    }
)

RPC_ds[['ITEM_CLASS_NAME', 'DEMOGRAPHIC_SEGMENT', 'SEGMENT_SALES', 'SEGMENT_SALES_SHARE', 'SALES_INDEX']]

,ITEM_CLASS_NAME,DEMOGRAPHIC_SEGMENT,SEGMENT_SALES,SEGMENT_SALES_SHARE,SALES_INDEX
0,500 RODENT POISON,Homeowner older kids,511851.98,0.077667,0.937367
1,500 RODENT POISON,Homeowner younger kids,877409.75,0.133135,0.838635
2,500 RODENT POISON,Older homeowner,3843724.49,0.583233,1.146715
3,500 RODENT POISON,Renter no kids,310016.32,0.047041,0.726408
4,500 RODENT POISON,Renter with kids,280581.77,0.042574,0.701336
5,500 RODENT POISON,Unclassifiable,64670.81,0.009813,1.130103
6,500 RODENT POISON,Younger homeowner,702125.23,0.106538,0.921335
7,500 RODENT REPELLENT,Homeowner older kids,63782.39,0.078380,0.945981
8,500 RODENT REPELLENT,Homeowner younger kids,126008.45,0.154848,0.975408
9,500 RODENT REPELLENT,Older homeowner,387861.45,0.476631,0.937121


- Renters under indexed in Poison, but over indexed in Repellent and Trap. 
- Homeowners with kids slightly under indexed in Poison, while Homeowners w/o kids slightly over indexed.
- Homeowners with kids slightly over indexed in Trap, while Homeowners w/o kids slightly under indexed.
- All homeowners slightly under indexed in Repellent.

## Commercial Industry Segment (NZ)

Class index

In [39]:
# Get data
RPC_is = con.read_sql_file_into_pandas(
    file="sql/5-commercial-industry-segment.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'n_commercial_segments': 10, 
        'table_name': table,
        'level' : 'item_class_name',
        'country' : 'NZ' 
    }
)

In [46]:
RPC_is.head(5)

,ITEM_CLASS_NAME,INDUSTRY_SEGMENT,SEGMENT_SALES,SEGMENT_QUANTITY,SEGMENT_CUSTOMERS,TOTAL_SALES,TOTAL_QUANTITY,TOTAL_CUSTOMERS,TOTAL_SEG_SALES,TOTAL_SEG_QUANTITY,...,OVERALL_CUSTOMERS,SEGMENT_SALES_SHARE,SEGMENT_QUANTITY_SHARE,SEGMENT_CUSTOMER_SHARE,OVERALL_SALES_SHARE,OVERALL_QUANTITY_SHARE,OVERALL_CUSTOMER_SHARE,SALES_INDEX,QUANTITY_INDEX,CUSTOMER_INDEX
0,500 RODENT POISON,Accommodation and Food Services,11979.36,533.0,363,173439.62,8089.0,4358,24786.70,1435.0,...,6776,0.069069,0.065892,0.083295,0.072145,0.070801,0.084120,0.957371,0.930661,0.990193
1,500 RODENT POISON,Carpentry Services,2816.62,143.0,95,173439.62,8089.0,4358,5646.09,332.0,...,6776,0.016240,0.017678,0.021799,0.016434,0.016381,0.022727,0.988202,1.079230,0.959168
2,500 RODENT POISON,Cleaning Services,3699.83,227.0,147,173439.62,8089.0,4358,9102.01,615.0,...,6776,0.021332,0.028063,0.033731,0.026493,0.030343,0.032468,0.805211,0.924840,1.038900
3,500 RODENT POISON,Commercial Builder,5559.59,245.0,159,173439.62,8089.0,4358,12200.06,644.0,...,6776,0.032055,0.030288,0.036485,0.035510,0.031774,0.035567,0.902706,0.953227,1.025810
4,500 RODENT POISON,Electrical Services,7364.10,377.0,268,173439.62,8089.0,4358,16241.78,993.0,...,6776,0.042459,0.046607,0.061496,0.047274,0.048993,0.063902,0.898155,0.951280,0.962349


In [41]:
total_summary = RPC_is.groupby('ITEM_CLASS_NAME').agg(
    TOTAL_SALES_ALL_SEGMENTS=('TOTAL_SALES', 'first'),  # consistent within class
    TOTAL_QUANTITY_ALL_SEGMENTS=('TOTAL_QUANTITY', 'first')
).reset_index()
total_summary

,ITEM_CLASS_NAME,TOTAL_SALES_ALL_SEGMENTS,TOTAL_QUANTITY_ALL_SEGMENTS
0,500 RODENT POISON,173439.62,8089.0
1,500 RODENT REPELLENT,24997.84,550.0
2,500 RODENT TRAP,145131.35,11629.0


In [47]:
RPC_is[['ITEM_CLASS_NAME', 'INDUSTRY_SEGMENT', 'SEGMENT_SALES', 'SEGMENT_SALES_SHARE', 'SALES_INDEX']]

,ITEM_CLASS_NAME,INDUSTRY_SEGMENT,SEGMENT_SALES,SEGMENT_SALES_SHARE,SALES_INDEX
0,500 RODENT POISON,Accommodation and Food Services,11979.36,0.069069,0.957371
1,500 RODENT POISON,Carpentry Services,2816.62,0.016240,0.988202
2,500 RODENT POISON,Cleaning Services,3699.83,0.021332,0.805211
3,500 RODENT POISON,Commercial Builder,5559.59,0.032055,0.902706
4,500 RODENT POISON,Electrical Services,7364.10,0.042459,0.898155
5,500 RODENT POISON,Landscaper and Gardening Services,6649.08,0.038337,0.929019
6,500 RODENT POISON,Manufacturing,27341.71,0.157644,0.941826
7,500 RODENT POISON,Professional Computer and Scientific Services,50952.77,0.293778,1.066813
8,500 RODENT POISON,Residential Builder,26770.53,0.154351,0.987158
9,500 RODENT POISON,Retail and Wholesale Trade,30306.03,0.174735,1.080208


## Drop item range table

In [43]:
# Drop item range table
con.execute_statement_from_sql_string(
    statement='DROP TABLE bdwprd_de.ia_merch_de.{table_name};',
    formatting={'table_name': table}
    )

# Brand view

In [44]:
overall_brand = con.read_sql_file_into_pandas(
    file="sql/Overall_by_brand.sql",
    formatting={
        "start_date":                 start_date,
        "end_date":                   end_date,
        "sub_dept":                   "'400 RODENT PEST CONTROL'"        
    })

In [45]:
overall_brand

,CUSTOMER_TYPE_CODE,BRAND_NAME,SALES,TOTAL_TRX,TOTAL_UNITS,NUM_ITEMS_PURCHASED,OVERALL_SALES,OVERALL_TRX
0,Consumer,KIWICARE,695580.01,31730,37769.0,12,940093.19,39491
1,Consumer,BIG CHEESE,978429.28,40915,51017.0,26,1250863.29,50286
2,Consumer,CLIX,0.22,1,1.0,1,0.22,1
3,Commercial,KIWICARE,244513.18,7761,9740.0,12,940093.19,39491
4,Consumer,TRAPPED!,303072.71,25195,44826.0,10,382278.67,30189
5,Commercial,NOOSKI,5341.51,337,457.0,3,26334.23,1752
6,Consumer,NO,6960.74,365,367.0,1,8275.91,437
7,Commercial,GOTCHA,11099.77,1278,2066.0,3,63059.13,7913
8,Consumer,NOOSKI,20992.72,1415,1747.0,3,26334.23,1752
9,Commercial,BIG CHEESE,272434.01,9371,12437.0,26,1250863.29,50286
